In [ ]:
# get final titles afters union or filter
def get_show_names(Anchor_Name,Pull_Year):
  store_US_titles = []

  store_nconst = get_nconst(Anchor_Name,Pull_Year)

  if store_nconst[0] == NONE:
    return None

  titlecrew = pd.read_table(TITLE_CREW)
  nconst_in_dir = titlecrew[titlecrew.directors.str.contains(store_nconst[0])]
  nconst_in_writer = titlecrew[titlecrew.writers.str.contains(store_nconst[0])]
  tconst_in_crew = list(set().union(nconst_in_dir.tconst.tolist(),nconst_in_writer.tconst.tolist()))

  # Combine_list_1 = list(set().union(store_nconst[1],get_all_shows(store_nconst[0])))
  Combine_list_2 = set().union(store_nconst[1],get_all_shows(store_nconst[0]),tconst_in_crew)

# Here applying US(region) filter in the shows

  titleakas = pd.read_table(TITLE_AKAS, sep='\t')
  titleakas.region = titleakas.region.fillna(REGION_TYPE)
  filter_region = titleakas[titleakas.region == REGION_TYPE]
  filter_region_titleid = filter_region.titleId.tolist()
  for x in Combine_list_2:
    if x in filter_region_titleid:
      store_US_titles.append(x)

  # return store_US_titles
  tConsts= set(store_US_titles)

  chunksize = 6*10**6

  titlebasics = pd.read_table(TITLE_BASICS, chunksize = chunksize)
  Show_Names = pd.DataFrame(columns=['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres'])
  for i,chunk in enumerate(titlebasics):
    show_names = chunk[chunk['tconst'].isin(tConsts)]
    # show_names = chunk[chunk['tconst'] == 'tt2788528']
    print('{} shows in chunk {}'.format(len(show_names), i))
    Show_Names = Show_Names.append(show_names, ignore_index = True)
    # print(Show_Names)
    filter_title_type = Show_Names[Show_Names['titleType']==TITLE_TYPE]
    filter_genres = filter_title_type[filter_title_type.genres.str.contains(GENRE_TYPES)]
    filter_end_year = filter_genres[filter_genres.endYear >= Pull_Year]
    filter_start_year = filter_end_year[filter_end_year.startYear.apply(str) <=Pull_Year]
  return filter_start_year 

In [ ]:
def final_show(anchor_list):
  X = {} #dict contains show name and their votes
  for host in anchor_list:
    print(host)
    store_show_names = get_show_names(host,'2006')
    if store_show_names is None:
      title = SHOW_NOT_IN_IMDB
      if title in X:
        X[title] +=1
      else:
        X[title] = 1
    
    else:
      for title in store_show_names.primaryTitle.tolist():
        if title in X:
          X[title] +=1
        else:
          X[title] = 1
  if bool(X) is False:
    return NONE
  else:
    Max_votes = max(X.values())
    # print(Max_votes)
    Majority_Shows = [title for title, votes in X.items() if votes==Max_votes]
    if len(Majority_Shows) is 1:
      print(Majority_Shows)
      # return Majority_Shows
    else:
      Open_Subtitles = open("/content/drive/My Drive/Datasets/Subtitles/2006-01-04_0000_US_00000063_V2_MB7_VHS7_H11_MS.txt3")
      Read_Subtitles = Open_Subtitles.read()

      for show in Majority_Shows:
        if Read_Subtitles.find(show.upper())!=-1:
          print(show)
          return show
          

      for show in Majority_Shows:
        for network in get_majority_network(anchor_list):
          if show.find(network)!=-1:
            print(show)
            return show
      
      # for show in Majority_Shows:
      #   call_channel = get_channel(show)
      #   if call_channel:
      #     for network in call_channel:
      #       if network == ('NBC'):
      #         return show 

        # else:
        #   return "network is not available"
